# Ajuste do circuito RLC com R = 600Ω

In [49]:
import pandas as pd
import altair as alt

### Leitura dos dados de tensão obtidos através do osciloscópio

In [50]:
p4c_data = pd.read_csv('./p4c_data0.csv')
p4c_data.head()

,Time(s),CH1V,CH2V
0,-0.005000,-0.011733,0.013600
1,-0.004995,-0.002933,0.021333
2,-0.004990,0.010400,0.022400
3,-0.004985,-0.010667,0.013600
4,-0.004980,0.019200,0.026933


Setando o tempo 0:

In [51]:
p4c_data["Time(s)"] = p4c_data["Time(s)"] + 5.000000e-03
p4c_data.head()

,Time(s),CH1V,CH2V
0,0.000000,-0.011733,0.013600
1,0.000005,-0.002933,0.021333
2,0.000010,0.010400,0.022400
3,0.000015,-0.010667,0.013600
4,0.000020,0.019200,0.026933


### Visualização inicial dos dados

In [62]:
p4c_CH1V = alt.Chart(p4c_data).mark_line(color='orange').encode(
    x = "Time(s)",
    y = "CH1V",
    tooltip = "Time(s)"
).interactive()

p4c_CH2V = alt.Chart(p4c_data).mark_point().encode(
    x = alt.X("Time(s)", title="Tempo (s)"),
    y = alt.Y("CH2V", title="Tensão (V)"),
    tooltip = "Time(s)"
).properties(title='Tensões medidas').interactive()

p4c_chart = p4c_CH1V + p4c_CH2V
p4c_chart

alt.LayerChart(...)

### Selecionando uma oscilação

In [54]:
p4c = p4c_data[(p4c_data["Time(s)"] >= 0.00193) & (p4c_data["Time(s)"] <= 0.002435)]
p4c_chart = alt.Chart(p4c).mark_point().encode(
    x = alt.X("Time(s)", scale=alt.Scale(domain=[0.00193, 0.002435])),
    y = alt.Y("CH2V"),
    tooltip = ["Time(s)", "CH2V"]
).interactive()
p4c_chart

alt.Chart(...)

### Configurando o fit para o ajuste 

In [55]:
import numpy as np
from scipy.optimize import curve_fit

# Função do modelo para o ajuste
def oscilacao_amortecida(t, Voff, A, alpha, wp, t0):
    return Voff + A * np.exp(-alpha * (t - t0)) * np.cos(wp * (t - t0))

### Ajustando a oscilação selecionada

In [65]:
# Dados experimentais
tempo = p4c['Time(s)'].to_numpy()
tensao_ruidosa = p4c['CH2V'].to_numpy()

# Ajuste dos parâmetros usando curve_fit
p0 = [0, 2.5, 69280, 131270, 0.00193]  # Estimativas iniciais para os parâmetros
params, covariance = curve_fit(oscilacao_amortecida, tempo, tensao_ruidosa, p0=p0)

# Parâmetros ajustados
Voff, A, alpha, wp, t0 = params
param_incertezas = np.sqrt(np.diag(covariance))
print(f"Parâmetros ajustados:")
print(f"Voff = {Voff:.4f} V | Incerteza: {param_incertezas[0]}")
print(f"A = {A:.4f} V | Incerteza: {param_incertezas[1]}")
print(f"alpha = {alpha:.4f} s^-1 | Incerteza: {param_incertezas[2]}")
print(f"wp = {wp:.4f} rad/s | Incerteza: {param_incertezas[3]}")
print(f"t0 = {t0:.4f} s | Incerteza: {param_incertezas[4]}")

# Calcular a frequência natural (omega_0) a partir de wp e alpha
omega_0 = np.sqrt(wp**2 + alpha**2)
print(f"Frequência natural (omega_0) = {omega_0:.4f} rad/s")

# Gerar dados ajustados para comparação
tensao_ajustada = oscilacao_amortecida(tempo, *params)

# Criar dataframe para visualização
df = pd.DataFrame({
    'Tempo (s)': tempo,
    'Tensão Medida (V)': tensao_ruidosa,
    'Tensão Ajustada (V)': tensao_ajustada,
    'nome_da_curva_1': 'Dados experimentais',
    'nome_da_curva_2': 'Curva ajustada'
})

# Visualização com Altair
chart = alt.Chart(df).mark_point().encode(
    x='Tempo (s)',
    y=alt.Y('Tensão Medida (V)', title='Tensão (V)', axis=alt.Axis(grid=True)),
    tooltip=['Tempo (s)', 'Tensão Medida (V)']
).properties(title='Ajuste de Oscilação Amortecida').interactive()

ajuste_chart = alt.Chart(df).mark_line(color='red').encode(
    x='Tempo (s)',
    y='Tensão Ajustada (V)',
    color=alt.Color('nome_da_curva_2', legend=alt.Legend(title="", orient="top-right"), scale=alt.Scale(range=['red']))
)

ajuste_chart + chart


Parâmetros ajustados:
Voff = 0.0187 V | Incerteza: 0.0009068955084345591
A = 3.1591 V | Incerteza: 0.01811845323908867
alpha = 83229.2402 s^-1 | Incerteza: 680.9388572108726
wp = 85710.5260 rad/s | Incerteza: 400.26872426995175
t0 = 0.0019 s | Incerteza: 7.650291847464824e-08
Frequência natural (omega_0) = 119471.3384 rad/s


alt.LayerChart(...)